# RAG_Flask_Ollama

This notebook will demonstrate a step-by-step procedure of creating a Retrieval-Augmented Generation (RAG) pipeline using LangChain and locally deploying the LLM application using Flask and Ollama. The data set used in this notebook consist of pdf files of research articles on the topic of AI.

In [1]:
#Import the necessary libraries

import numpy as np
import sys
import langchain
import langchain_community
import langchain_core
import langchain_openai
import sentence_transformers
import pypdf
import dotenv
import importlib.metadata

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

print('Version information')

print('python: {}'.format(sys.version))
print('numpy: {}'.format(np.__version__))
print('langchain: {}'.format(langchain.__version__))
print('langchain_community: {}'.format(langchain_community.__version__))
print('sentence_transformers: {}'.format(sentence_transformers.__version__))
print('pypdf: {}'.format(pypdf.__version__))
print('langchain_core: {}'.format(langchain_core.__version__))
print('langchain_openai: {}'.format(importlib.metadata.version("langchain-openai")))
print('python-dotenv: {}'.format(importlib.metadata.version("python-dotenv")))
print('ollama: {}'.format(importlib.metadata.version("ollama")))

c:\Users\ohjin\anaconda3\envs\rag_flask_ollama\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Version information
python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
numpy: 1.24.3
langchain: 0.2.14
langchain_community: 0.2.12
sentence_transformers: 3.0.1
pypdf: 4.3.1
langchain_core: 0.2.43
langchain_openai: 0.1.25
python-dotenv: 1.1.1
ollama: 0.5.3


## 1) Retrieval Augmented Generation (RAG)

State-of-the-art Large Language Models (LLM's) are trained on an enormous corpus of textual data (taken from webpages, articles, books etc.), and they store a wide range of general knowledge in their parameters. While they are able to perform well on tasks that require general knowledge, they tend to struggle on tasks that require information that wasn't present in the training data. For instance, LLM's may struggle on tasks that require knowledge of domain-specific information, or even up-to-date information.

It is very important to overcome this problem, because it is undesirable to get a non-answer from the LLM, and potentially even dangerous if the LLM begins to hallucinate (i.e ramble on with an answer that seems believable but is factually inaccurate). Therefore, it is essential to bridge the gap between the LLM's general knowledge, and other domain-specific or up-to-date information in order to help the LLM generate responses that are contextual and factually accurate, while reducing the chances of hallucinations.

There are two effective ways of accomplishing this:
1. <strong>Fine-tuning the LLM on the domain-specific/proprietary/new data</strong>:
    <ul>
        <li>By fine-tuning the model, it can be made suitable for the task.</li>
        <li>However, this comes with some limitations. It is compute-intensive, expensive and not agile (it's not realistic to fine-tune an LLM with the new data coming in everyday)</li>
    </ul>
2. <strong>Retrieval Augmented Generation (RAG)</strong>:
    <ul>
        <li><a href='https://arxiv.org/abs/2005.11401'>This technique</a> provides the LLM with contextual information from an external knowledge source that can be updated more easily.</li>
        <li>It allows the LLM to generate more contextual, and factually accurate responses by allowing it to dynamically access information from an external knowledge source.</li>
    </ul>

<center><img src="data/images/rag-architecture.png" alt="drawing" width="700" align='center'></center>
<center>Basic RAG Architecture</center>


The RAG pipeline consists of the following steps:
<ol>
    <li><strong>Retrieval</strong>: The user's query is used to retrieve the relevant contextual information from the external knowledge source. The external knowledge source is a vector store that contains the embeddings of the documents that contain the proprietary/domain-specific data. The user query is embedded into the same vector space as these documents, and a similarity search is performed in this embedding space to retrieve the documents that are most similar to the user's query. These retrieved documents make up the context that the LLM will consider in order to produce factually accurate responses. </li>
    <li><strong>Augmentation</strong>: The user's query is augmented with the retrieved contextual information to form the prompt. The prompt will also usually include instructions to the LLM for performing the task. There is an entire sub-field known as Prompt Engineering, that is dedicated to fine-tuning the prompt so as to get the best possible response from the LLM, and you will get some experience with it in <strong>7.2</strong> while implementing the RAG Chain. </li>
    <li><strong>Generation</strong>: The constructed prompt, including the instructions to the LLM, the user query and the retrieved context, is fed to the LLM to generate a response that is comprehensible and factually correct.</li>
</ol>

## 1.1) Implementing the Retriever

In the first step, the Retriever for the RAG Chain will be implemented. The documents serving as the basis of the external knowledge source will be split into smaller chunks to be used with the vector database. Lastly, the Retriever object will be created to be used with the RAG Chain to retrieve relevant document chunks as additional contextual information.

### 1.1.1) Loading and Pre-processing data: Document Loaders and Text Splitters

Langchain provides <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/'>several classes</a> to load data into Document objects. There are classes to load data from HTML files, PDF files, JSON files, CSV files, file directories etc. Here, Langchain's <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#pypdf-directory'>PyPDFDirectoryLoader</a> will be used to load PDF's from a directory. A Document object is a dictionary that stores the text and metadata about each document.

Once the data has been loaded into Document(s), it's common to split the documents into smaller chunks. This is done because when a user inputs a query to the system, the retriever will return the most relevant documents, which will be augmented to the prompt that is sent to the LLM. There are limits to the length of this prompt, since it must fit into the LLM's context window. Therefore, it's common to split documents into smaller chunks, so that only the retrieved relevant chunks are inserted into the prompt. Langchain offers several <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/'>TextSplitters</a> to perform the document splitting. RecursiveCharacterTextSplitter, which is a way to split a document into several chunks in a manner such that related pieces of text are kept together in a chunk, will be used here.

In [2]:
from retriever import Retriever
from local_tests.retriever_test import Retriever_Test

local_test = Retriever_Test()
stu_retriever = Retriever()
num_chunks_to_query = 2

print('Local Tests for Loading and Splitting Documents \n')

# Local test for load documents
output_documents = stu_retriever.loadDocuments(data_dir='./data/papers/')
load_test = (len(output_documents) == local_test.load_documents_len)
print('Your load documents works as expected:', load_test)

# Local test for split documents
output_chunks = stu_retriever.splitDocuments(output_documents, chunk_size=700, chunk_overlap=50)
split_test = (len(output_chunks) == local_test.split_documents_len)
print('Your split documents works as expected:', split_test)

Local Tests for Loading and Splitting Documents 

Your load documents works as expected: True
Your split documents works as expected: True


In [3]:
# Load the PDF documents
from retriever import Retriever
retriever = Retriever()
documents = retriever.loadDocuments(data_dir='./data/papers/')

print('documents type:', type(documents))           # python list
print('documents[0] type:', type(documents[0]))     # Document object
print('documents length:', len(documents))          # each page is loaded as a separate document (104 pages -> 104 documents)

print('\nContent of documents[0]:\n', documents[0]) # the first document object, containing 'page_content' and 'metadata' fields

documents type: <class 'list'>
documents[0] type: <class 'langchain_core.documents.base.Document'>
documents length: 104

Content of documents[0]:
 page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. W

In [4]:
# Split the documents into smaller chunks
document_chunks = retriever.splitDocuments(documents)

print('document_chunks type:', type(document_chunks)) # python list
print('document_chunks[0] type:', type(document_chunks[0])) # Document object
print('document_chunks length:', len(document_chunks)) # you should observe that each document (corresponding to a page in a PDF) has been split into several chunks

for chunk in document_chunks[:3]:   # displaying the first 3 chunks
    print('\n\n', chunk)

document_chunks type: <class 'list'>
document_chunks[0] type: <class 'langchain_core.documents.base.Document'>
document_chunks length: 701


 page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or' metadata={'source': 'data\\papers\\attention.pdf', 'page': 0}


 page_content='convolutional neural networks that include an encoder and a decoder. The best
performing mod

In [5]:
# Analysis of chunk size
def compute_avg_chunk_size(chunks):
    return sum([len(chunk.page_content) for chunk in chunks])/len(chunks)

print(f'Before split, there were {len(documents)} documents, with average size equal to {compute_avg_chunk_size(documents)}.')
print(f'After split, there were {len(document_chunks)} documents (chunks), with average size equal to {compute_avg_chunk_size(document_chunks)}.')

Before split, there were 104 documents, with average size equal to 4149.2307692307695.
After split, there were 701 documents (chunks), with average size equal to 624.5235378031384.


### 1.1.2) Creating the Vector Database and Retrieval System: Embedding models and Vector Stores

<a href='https://python.langchain.com/v0.2/docs/integrations/text_embedding/'>Langchain provides several embedding models</a>. An embedding model converts a piece of natural language (e.g. a token) into a vector embedding. The next steps will involve using Huggingface's BGE Embedding models, which are the one of the best open-source embedding models <a href='https://python.langchain.com/v0.2/docs/integrations/text_embedding/bge_huggingface/'>(according to Langchain)</a>. The model used (BAAI/bge-small-en-v1.5) has 384-dimensional embedding vectors.

After obtaining the embedding model, vectorstore needs to be created by embedding all of the document chunks into the vector space. A retriever will be used to retrieve the document chunks from the vectorstore that are most similar to the user's query using efficient similarity search algorithms. Langchain offers <a href='https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/'>several vectorstores</a>, and Facebook's AI Similarity Search (FAISS) will be used.

In [6]:
from retriever import Retriever
from local_tests.retriever_test import Retriever_Test

local_test = Retriever_Test()
stu_retriever = Retriever()
num_chunks_to_query = 2

print('Local Tests for Retriever \n')   # ensure that the local tests for the loading and splitting documents pass

output_documents = stu_retriever.loadDocuments(data_dir='./data/papers/')
output_chunks = stu_retriever.splitDocuments(output_documents, chunk_size=700, chunk_overlap=50)

# Local test for retriever embeddings
output_retrieval_system = stu_retriever.createRetriever(output_chunks, num_chunks_to_return=num_chunks_to_query)
student_embedding_model = stu_retriever.huggingface_embeddings
output_embedding = np.array(student_embedding_model.embed_query(output_chunks[0].page_content))
embedding_shape = output_embedding.shape[0]

if (student_embedding_model.model_name == local_test.model_name):
    embedding_shape_test = (embedding_shape == local_test.embedding_size)
    print('Your retriever embeddings returns the expected shape:', embedding_shape_test)
else:
    print('You are free to choose the embedding model to use for the best results (provided it works with Gradescope),')
    print(f'but we can only locally test the embedding shape (no value testing) if using the {local_test.model_name} model')
    print('and the RecursiveCharacterTextSplitter.')

# Local test for chunk relevance
output_retrieved_chunks = output_retrieval_system.invoke(local_test.relevance_prompt)
returned_count_test = (len(output_retrieved_chunks) == num_chunks_to_query)
relevance_test = True
for chunk in output_retrieved_chunks:
    if local_test.relevant_pdf not in chunk.metadata['source']:
        relevance_test = False
print('Your retriever returns the expected number of chunks:', returned_count_test)
print('Your retriever returns chunks from the expected relevant documents:', relevance_test)

Local Tests for Retriever 

Your retriever embeddings returns the expected shape: True
Your retriever returns the expected number of chunks: True
Your retriever returns chunks from the expected relevant documents: True


In [7]:
# Create the retriever
retrieval_system = retriever.createRetriever(document_chunks)
embedding_model = retriever.huggingface_embeddings

In [8]:
# Sample embedding for a document chunk
sample_embedding = np.array(embedding_model.embed_query(document_chunks[0].page_content))
print("Size of the embedding: ", sample_embedding.shape)
print("Sample embedding of a document chunk: ", sample_embedding)

Size of the embedding:  (384,)
Sample embedding of a document chunk:  [-4.52586636e-02 -4.26387973e-03 -4.41800393e-02 -2.71606948e-02
 -3.99877205e-02  3.49567235e-02 -3.55790905e-03  1.92805864e-02
  6.00379892e-02 -8.17402080e-03  2.22530942e-02 -1.37564540e-02
  4.14049439e-02  3.60869430e-02  1.50960404e-02 -2.18757540e-02
 -5.24056070e-02  3.74929532e-02  9.44254268e-03 -4.80045285e-03
  6.43263608e-02 -4.49812412e-02 -7.86372460e-03  1.68564124e-03
  2.98901144e-02  4.22352692e-03 -7.87110534e-03 -2.22316664e-02
 -2.05486305e-02 -2.44379282e-01  1.11250076e-02 -3.14529017e-02
  5.13514206e-02  2.52507143e-02 -1.31557165e-02 -2.39112619e-02
 -5.89635931e-02 -2.62947343e-02 -6.48454800e-02  3.48604023e-02
  1.73388626e-02  2.02526003e-02  1.45917793e-03 -1.66223999e-02
 -1.07647590e-02 -2.87664086e-02 -4.01396714e-02 -4.46370542e-02
 -4.39290777e-02 -3.29201715e-03 -5.14779538e-02  8.32557492e-03
 -4.71756868e-02  8.11673328e-02  2.99861450e-02 -3.70062049e-03
  3.59496996e-02  2.

In [9]:
# Demonstration of the retriever finding the relevant document chunks
questions = [
    "What novel techniques did the 'Attention is all you need' paper introduce?",
    "List the metrics were used to compare GloVe vectors with other embedding methods such as Word2Vec?"
]

for question in questions:
    print(f'QUESTION: {question}' + '\n')

    retrieved_chunks = retrieval_system.invoke(question)
    print('RETRIEVED CHUNKS: ')
    for chunk in retrieved_chunks:
        print(chunk, '\n')

    print('\n\n')

QUESTION: What novel techniques did the 'Attention is all you need' paper introduce?

RETRIEVED CHUNKS: 
page_content='2020. URL https://arxiv.org/abs/2004.14366 .
[58] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
Ł ukasz Kaiser, and Illia Polosukhin. Attention is all you need. In I. Guyon, U. V . Luxburg,
S. Bengio, H. Wallach, R. Fergus, S. Vishwanathan, and R. Garnett, editors, Advances in Neural
Information Processing Systems 30 , pages 5998–6008. Curran Associates, Inc., 2017. URL
http://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf .
[59] Ashwin Vijayakumar, Michael Cogswell, Ramprasaath Selvaraju, Qing Sun, Stefan Lee, David
Crandall, and Dhruv Batra. Diverse beam search for improved description of complex scenes.' metadata={'source': 'data\\papers\\rag.pdf', 'page': 14} 

page_content='networks , vol. 3361, no. 10, p. 1995, 1995.
[18] A. Vaswani, N. Shazeer, N. Parmar, J. Uszkoreit, L. Jones, A. N.
Gomez, Ł. Kaiser, and I. Po

## 1.2) Implementing the RAG Chain

Building applications with Large Language Models (LLM's) requires permissions from the LLM provider via an API key. There are two types of LLM models available for use:
<ol>
    <li>Open-source models, which are usually smaller models with lesser capabilities (e.g. Llama created by FaceBook, Flan-T5 created by Google)</li>
    <li>Proprietary models, which are usually larger with better performace (e.g. GPT-4o by OpenAI, Gemini-1.5 by Google etc.), but aren't free to use.</li>
</ol>

A free-to-use, open-source HuggingFace LLM will be used here. In order to use a HuggingFace LLM, there are some steps that first need to completed:
<ul>
    <li>Create a <a href= "https://huggingface.co/">Huggingface</a> Account</li>
    <li>Create a new <a href='https://huggingface.co/settings/tokens'>API Access Token</a> with the <strong>WRITE</strong> Token Type. <strong>Save this access token in a secure place and do not share it with others.</strong> <strong>Save the token to a <a href="https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1">.env file</a>.</strong> </li>
    <li>Accept the terms and conditions. This step may be required for certain models like <a href= 'https://huggingface.co/mistralai/Mistral-7B-v0.1'>mistralai/Mistral-7B-v0.1</a>. View the status of the model request <a href='https://huggingface.co/settings/gated-repos'>here</a></li>
</ul>

The model must also be hosted on HuggingFace Spaces. To do this:
<ul>
  <li>Go to: <a href= https://huggingface.co/spaces>Spaces</a></li>
  <li>Click "+ New Space" in the top right</li>
  <li>Fill in:
    <ul>
      <li>Space name: Choose any</li>
      <li>SDK: Choose Gradio (blank template)</li>
      <li>Space hardware: Choose CPU basic (free)</li>
      <li>Visibility: Private</li>
    </ul>
  </li>
  <li>Click: Create Space</li>
  <li>Select Files in the top right which will direct to a new page</li>
  <li>Select + Contribute in the top right</li>
  <li>Select Upload files</li>
  <li>Add the files in hf_spaces folder provided to user space, modifying them appropriately for the models</li>
</ul>

The .env file will need the following:
* Set `HUGGINGFACE_API_KEY`: Obtain the Huggingface API key
* Set `GRADIO_SPACE_NAME`: Set the Gradio space name after creating it

### 1.2.1) Choosing an LLM and Initalizing the Retriever System

In [10]:
# !cat .env
# !rm .env

In [14]:
from rag_chain import RAG_Chain
from local_tests.retriever_test import Retriever_Test
from langchain_core.vectorstores.base import VectorStoreRetriever

# Instantiate + configure the RAG_Chain class to use a HF-hosted Gradio Space via a custom LLM wrapper
rag_chain = RAG_Chain(data_dir='./data/papers/', llm_type="gradio_flan")
local_test = Retriever_Test()

# Print the LLM used
print(rag_chain.llm)

# Local test for RAG retriever system

# Check that retriever is of expected type:
rag_retriever = rag_chain.retriever_system
type_test = isinstance(rag_retriever, VectorStoreRetriever)
print('Your retriever is of type VectorStoreRetriever:', type_test)

# Check that the retriever retrieves relevant chunks
output_retrieved_chunks = rag_retriever.invoke(local_test.relevance_prompt)
relevance_test = True
for chunk in output_retrieved_chunks:
    if local_test.relevant_pdf not in chunk.metadata['source']:
        relevance_test = False
print('Your retriever returns chunks from the expected relevant documents:', relevance_test)

Loaded as API: https://ojw92-cs8803-hw4.hf.space ✔
GradioLLMWrapper
Params: {}
Your retriever is of type VectorStoreRetriever: True
Your retriever returns chunks from the expected relevant documents: True


### 1.2.2) Formatting the Prompt with PromptTemplates

Prompts are a set of instructions that are given to an LLM in order to guide it to produce responses that are coherent, contextual and relevant. It usually takes several edits and changes to the prompt until the LLM produces the desirable response. This process is called Prompt Engineering. It is common practice to save a good, desirable prompt as a template for any time this prompt is needed to be used again. This is done with PromptTemplates.

The **createPrompt** function in **rag_chain.py** takes as an input parameter, a dictionary that stores a question along with 4 possible answer choices. For example, the input parameter could look like:

{ <br>
   &emsp; 'question': "What is the main contribution of the Transformer architecture?", <br>
   &emsp; 'A': "It introduces convolutional layers for sequence tasks.", <br>
   &emsp; 'B': "It improves word embeddings using context.", <br>
   &emsp; 'C': "It removes recurrence and uses self-attention mechanisms.", <br>
   &emsp; 'D': "It uses RNNs for language modeling." <br>
}

In [15]:
# Print the empty prompt template
prompt_template = rag_chain.createPrompt(question={'question': "", "A": "", "B": "", "C": "", "D": ""})

print(prompt_template)

Human: Question: 
A: 
B: 
C: 
D: 
Please select the best answer and explain your choice.


In [16]:
# Print the formatted prompt with the question. This is what will be passed to the LLM.
from local_tests.rag_test import RAG_Test
tests = RAG_Test()

prompt_with_question = rag_chain.createPrompt(question=tests.question1)
print(prompt_with_question)

Human: Question: What is the main contribution of the Transformer architecture?
A: It introduces convolutional layers for sequence tasks.
B: It improves word embeddings using context.
C: It removes recurrence and uses self-attention mechanisms.
D: It uses RNNs for language modeling.
Please select the best answer and explain your choice.


### 1.2.3) Creating Chains

In LangChain, "chains" are a core concept designed to manage and streamline interactions with language models. They allow creating sequences of operations where the output of one step can be used as the input for the next. This is particularly useful for building complex workflows and applications that involve multiple stages of processing.

Langchain offers a RetrievalQA chain, which combines the retriever module with a QA chain (short for Question-Answering). The retriever is used to retrieve relevant documents from the vectorstore, and the QA chain answers questions based on the retrieved documents.

In [17]:
qa_chain = rag_chain.createRAGChain()

In [18]:
from local_tests.rag_test import RAG_Test

tests = RAG_Test()

print('Local Tests for end-to-end RAG pipeline', '\n\n')

questions = tests.local_test_questions
answers = tests.local_test_answers

correct = 0
total = len(questions)
for i in range(len(questions)):
    # Create the prompt with a question
    prompt_with_question = rag_chain.createPrompt(question=questions[i])
    print(prompt_with_question)

    # Query the LLM
    response = qa_chain(prompt_with_question)

    print('Answer selected by the LLM:', response['result'])
    print('Correct answer:', answers[i])

    if response['result'] == answers[i]:
        correct += 1

print(f'{correct}/{total} questions answered correctly')

Local Tests for end-to-end RAG pipeline 


Human: Question: What is the main contribution of the Transformer architecture?
A: It introduces convolutional layers for sequence tasks.
B: It improves word embeddings using context.
C: It removes recurrence and uses self-attention mechanisms.
D: It uses RNNs for language modeling.
Please select the best answer and explain your choice.
Answer selected by the LLM: C
Correct answer: C
Human: Question: The Transformer architecture relies heavily on which mechanism to process information?
A: Convolution
B: Self-attention
C: Pooling
D: Dropout
Please select the best answer and explain your choice.
Answer selected by the LLM: B
Correct answer: B
Human: Question: What is the primary innovation introduced by BERT?
A: Unidirectional language models
B: Bidirectional training of transformers
C: Convolutional embeddings
D: Memory networks
Please select the best answer and explain your choice.
Answer selected by the LLM: B
Correct answer: B
Human: Questio

## 2) Hosting and Deploying LLM and RAG

This section will explore Ollama-hosted and Flask Ollama-hosted LLMs within the RAG pipeline created in the previous question. The RAG system will be deployed using a Flask container, simulating a real-world LLM Pipeline deployment.

This section will compare different hosting methods:

- **Cloud-based LLMs** (Hugging Face Hub or Spaces) offload the burden of compute but require external API calls.
- **Local LLMs** (Ollama) offer greater control over data, cost, and customizations, but compute is limited by hardware.
- **Network-hosted LLMs** (Flask Ollama) allow for remote access within a private infrastructure, which is useful for on-premise deployments in industries like healthcare and finance where data privacy is critical.
  
In many real-world applications, LLMs are accessed through APIs rather than used directly. In section 1), Hugging Face’s API was used via Hugging Face Spaces. Now, the same RAG pipeline will be deployed but using Ollama and Flask.

The approach presented here is only one way to accomplish the task, but serves as an introduction and a vital opportunity to experiment with different deployment strategies.

## 2.1) Using Ollama with RAG

This section introduces Ollama and use the Ollama LLM in the RAG pipeline.

### 2.1.1) Getting Started with Ollama

First, [Ollama](https://ollama.com/) needs to be downloaded and the following model will need to be pulled:
- `llama3.2`
  
Refer to [ollama-python Documentation](https://github.com/ollama/ollama-python) (prerequisites section) and [Quickstart Guide](https://github.com/ollama/ollama/blob/main/README.md#quickstart) as necessary. Make sure the ollama server is running on the machine prior to running the local test below.

**Ollama must be kept running for the next sections to work.**

In [ ]:
# https://medium.com/@abonia/running-ollama-in-google-colab-free-tier-545609258453
# !ollama run gemma3
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

# curl https://ollama.ai/install.sh | sh
# ollama serve &        # start the server
# ollama pull llama3.2

In [ ]:
%env TOKENIZERS_PARALLELISM=(true | false)

In [ ]:
import pipeline as tap
from local_tests.deploy_test import Deploy_Test

local_test = Deploy_Test()

# Check that Ollama returns the expected response
response = tap.query_ollama(local_test.ollama_query)
response_check = response == local_test.ollama_response
print('Your Ollama server returned the expected response:', response_check)

### 2.1.2) Using Ollama with RAG

In [ ]:
from local_tests.deploy_test import Deploy_Test
from rag_chain import RAG_Chain

# Local test for RAG using Ollama LLM

# Check that Ollama works with rag_chain.py
rag_chain_oo = RAG_Chain(data_dir='./data/papers/', llm_type="ollama_only", init_retriever=False)
rag_chain_oo.llm.temperature = 0
local_test = Deploy_Test()

# Printing the LLM used
print(f"LLM Info:\n {rag_chain_oo.llm}\n")

# Check that RAG returns the expected response
response = rag_chain_oo.query_the_llm(local_test.ollama_rag_query)
response_check = response == local_test.ollama_rag_response
print('Your Ollama LLM in the RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.ollama_rag_query)
print("EXPECTED RESPONSE:", local_test.ollama_rag_response)
print(f"Your Response: {response}" if not response_check else "")

## 2.2) Deploying an LLM and using it with RAG

In the next steps, the Ollama LLM from 2.1.1) will be deployed by containerizing the LLM using Flask. While Ollama works locally, Flask can expose the LLM for network use. It is possible to query into this Flask Ollama LLM by reusing Langchain's OpenAI wrapper.

### 2.2.1) Using Flask to Containerize the Ollama LLM

In the terminal or command prompt, run the llm_app.py file before running the local test cell below.

**Keep llm_app.py running to complete the next section - do not terminate it.**

In [ ]:
# Run the llm_app.py file running in the background for the next test cell
# !python llm_app.py &
import subprocess

process = subprocess.Popen(['python', 'llm_app.py'])

In [ ]:
import pipeline as tap
from local_tests.deploy_test import Deploy_Test

local_test = Deploy_Test()

# Local test for Flask Ollama server

#Check that Ollama returns the expected response
response = tap.query_flask_ollama(local_test.ollama_query)
response_check = response == local_test.ollama_response
print('Your Flask Ollama server returned the expected response:', response_check)

### 2.2.2) Using Flask Ollama with RAG

In [ ]:
import pipeline as tap
from local_tests.deploy_test import Deploy_Test
from rag_chain import RAG_Chain

local_test = Deploy_Test()

# Local test for RAG using Flask Ollama

# Check that Ollama works with rag_chain.py
rag_chain_fo = RAG_Chain(data_dir='./data/papers/', llm_type="flask_ollama", init_retriever=False)
rag_chain_fo.llm.temperature = 0
local_test = Deploy_Test()

# Printing the LLM used
print(f"LLM Info:\n {rag_chain_fo.llm}\n")

# Check that RAG returns the expected response
response = rag_chain_fo.query_the_llm(local_test.flask_ollama_rag_query)
response_check = response == local_test.flask_ollama_rag_response
print('Your Flask Ollama LLM in the RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.flask_ollama_rag_query)
print("EXPECTED RESPONSE:", local_test.flask_ollama_rag_response)
print(f"YOUR RESPONSE: {response}" if not response_check else "")

## 2.3) Deploying RAG using Flask

### 2.3.1) Deploying RAG Local Test

In the terminal or command prompt, run the rag_app.py file before running the local test cell below.

**Keep rag_app.py running to complete the next section - do not terminate it.**

In [ ]:
process_llm = subprocess.Popen(['python', 'llm_app.py'])

In [ ]:
# Run the rag_app.py file running in the background for the next test cell
# !python rag_app.py &
import subprocess

process_rag = subprocess.Popen(['python', 'rag_app.py'])

In [ ]:
!python rag_app.py

In [ ]:
print("RESPONSE:", response)
print('================================')

In [ ]:
import requests

def is_flask_up(port):
    try:
        r = requests.get(f"http://127.0.0.1:{port}/")
        # Can check r.status_code or r.json()
        return True
    except Exception:
        return False

with open("rag_port.txt") as f:
    port = int(f.read().strip())

if is_flask_up(port):
    print(f"Flask server is running on port {port}!")
else:
    print(f"Flask server is NOT running on port {port}.")

In [ ]:
import pipeline as tap
from local_tests.deploy_test import Deploy_Test

local_test = Deploy_Test()

# Local test for Ollama server

# Check that flask rag returns the expected response
response = tap.query_flask_rag(local_test.flask_rag_query)
response_check = response[0]["text"] == local_test.flask_rag_response
print('Your Flask RAG system returned the expected response:', response_check)
print("\nQUERY:", local_test.flask_rag_query)
print("EXPECTED RESPONSE:", local_test.flask_rag_response)
print(f"YOUR RESPONSE: {response[0]['text']}" if not response_check else "")

### 2.3.2) Deploying RAG Final Test

In [ ]:
%%time

import pipeline as tap

tap.main()

**Running the pipeline should produce hashed_answers.json**